Centroids - most relevant tokens; tokens that contain the same meaning
1. Sum up vector representation of words that are part of a centroid => get embedding representation of the centroid.
2. Every sentence is scored (cosine similarity) based on how similar they are to the centroid embedding.
3. Select sentences based on their score until a certain number of words (hyperparameter) is reached
4. Avoid redundancy - if a chosen sentence is too similar to the ones in the already produced summary, don't add it (cosine similarity + predefined threshold)

https://aclanthology.org/W17-1003.pdf

https://arxiv.org/pdf/1707.02268v3.pdf

News headlines

Web snippets from search results

Below text is from https://pl.wikipedia.org/wiki/Avantasia

In [296]:
text = '''
Głównym bohaterem jest Gabriel Laymann, nowicjusz klasztoru dominikanów w Mainz. Akcja toczy się w roku 1602 roku gdzie wraz z resztą braci bierze udział w polowaniu na czarownice. Jednak niespodziewane spotkanie swojej przyrodniej siostry Anny Held, osądzanej o wykonywanie czarów, powoduje że Laymann odrzuca regułę klasztoru. Podstępnie zakrada się do biblioteki gdzie studiuje zakazane księgi. Zostaje przyłapany przez swojego nauczyciela brata Jakoba i uwięziony w lochu.

Główny bohater spotyka tam starszego człowieka Lugaida Vandroiy, który przedstawia mu się jako druid (Reach Out For The Light). Spotkany człowiek opowiada Gabrielowi o innym zagrożonym wymiarze – Avantasii. W zamian za pomoc w uratowaniu Avantasii druid obiecuje uratowanie Anny. Razem udaje im się uciec z lochów (Breaking Away) po czym Vandroiy zabiera Gabriela do kamieniołomu, gdzie ukryty jest portal łączący oba światy. Vandroiy używa go by przenieść Gabriela do równoległego świata.

W tym czasie biskup Mainz Johann Adam von Bicken, brat Jakob oraz rządca Falk von Kronberg są w drodze do Rzymu gdzie zamierzają spotkać się z papieżem Clemensem VIII (Glory of Rome). Niosą ze sobą także księgę odkrytą przez Gabriela. Wedle starożytnego zapisu wynika że księga jest ostatnią siódmą częścią pieczęci, która w całości daje właścicielowi absolutną wiedzę gdy tylko dostanie się on do wieży wyznaczającej środek Avantasii.

Gdy Gabriel dostaje się do równoległego świata jest powitany przez dwóch jego mieszkańców elfa Elderana oraz krasnoluda Regrina (Inside). Opowiadają mu o toczącej się wojnie przeciwko siłom zła oraz o planach papieża (Sign Of The Cross). Jeżeli papież użyje pieczęci połączenie między Avantasią a światem ludzi zostanie zamknięte, a oba światy dotkną straszliwe kataklizmy. Gabriel przybywa w momencie gdy Clemens VIII rozmawia z tajemniczym głosem dochodzącym z wieży. Zręcznemu Gabrielowi udaje się skraść pieczęć papieżowi po czym zanosi ją do miasta elfów (The Tower). Zdarzenie to kończy pierwszy album.

Jednak Gabriel nie jest usatysfakcjonowany. Chce dowiedzieć się więcej o świecie Avantasii dlatego Elderane wysyła nowicjusza do drzewa poznania. Tam Gabriel podczas objawienia widzi brata Jakoba który znosił okropny ból w jeziorze ognia (The Final Sacrifice). Elderane opowiada Gabrielowi o złotym kielichu ukrytym w rzymskich katakumbach. Kielich jest więzieniem dla ogromnej ilości torturowanych dusz, artefakt strzeżony jest także przez siejącą postrach bestię. Mimo niepowodzeń delfickich wypraw, Gabriel i Regrin powracają na ziemię by zmierzyć się z bestią. Przyjaciele znajdują kielich i przewracają go co umożliwia ucieczkę duszom. Przebudzona bestia zabija jednak krasnoluda, Gabrielowi udaje się uciec.

Gabriel wraca do Vandroiya, który czekał na niego. Druid spełnia obietnice, zakrada się do więzienia by uwolnić Annę. Jednak znajduje tam „przemienionego” brata Jakoba który także chce uwolnić Annę. Falk von Kronberg nakrywa ich i każe aresztować. Rozpoczęła się walka w której poległ Vandroiy raniony przez Kronberga, który później zostaje uśmiercony przez brata Jakoba. Anna ucieka by ponownie złączyć się z Gabrielem. Podążają wspólnie nieznaną drogą w przyszłość (Into The Unknown).
'''

In [336]:
import nltk
from nltk.corpus import stopwords
import numpy as np
import re
import string

from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from typing import List
from functools import reduce
import operator
from gensim.models import Word2Vec
from IPython.core.display import display, HTML

In [337]:
# STOP_WORDS = set(stopwords.words('english'))
from spacy.lang.pl.stop_words import STOP_WORDS

In [338]:
vector = List[float]

def dot(v: vector, w: vector):
    return sum([vi * wi for vi, wi in zip(v,w)])

def cos_sim(v: vector, w: vector):
    return dot(v, w) / (dot(v,v) * dot(w,w)) ** .5


In [339]:
class Preprocessing(object):
    def __init__(self, text):
        self.text = text
        self.oryg = text

    def lower(self):
        self.text = self.text.lower() 
        return self.text
    
    def remove_punctuation(self):
        self.text = self.text.translate(self.text.maketrans('', '', string.punctuation.replace('.', '')))
        return self.text 
    
    def remove_stop_words(self):
        self.text = ' '.join([word for word in self.text.split() if word not in STOP_WORDS])
        return self.text
    
    def remove_digits(self):
        self.text = re.sub(r'[\d+]', '', self.text)
        return self.text
    
    def sentence_tokenize(self):
        self.text = sent_tokenize(self.text)
        self.text = [sent.replace('.','') for sent in self.text]
        return self.text
    
    def basic_pipeline(self):
        self.lower()
        self.remove_digits()
        self.remove_punctuation()
        self.remove_stop_words()
        self.sentence_tokenize()
        return self.text

    def __call__(self):
        return self.text

In [340]:
def prepare_text(text):
    cleaned_text = Preprocessing(text)
    sentences = cleaned_text.basic_pipeline()
    raw_sentences = Preprocessing(text).sentence_tokenize()
    return sentences, raw_sentences

In [341]:
sentences, raw_sentences = prepare_text(text)

In [342]:
sentences

['głównym bohaterem gabriel laymann nowicjusz klasztoru dominikanów mainz',
 'akcja toczy wraz resztą braci bierze udział polowaniu czarownice',
 'niespodziewane spotkanie swojej przyrodniej siostry anny held osądzanej wykonywanie czarów powoduje laymann odrzuca regułę klasztoru',
 'podstępnie zakrada biblioteki studiuje zakazane księgi',
 'zostaje przyłapany swojego nauczyciela brata jakoba uwięziony lochu',
 'główny bohater spotyka starszego człowieka lugaida vandroiy przedstawia druid reach out for the light',
 'spotkany człowiek opowiada gabrielowi innym zagrożonym wymiarze – avantasii',
 'zamian pomoc uratowaniu avantasii druid obiecuje uratowanie anny',
 'razem udaje uciec lochów breaking away czym vandroiy zabiera gabriela kamieniołomu ukryty portal łączący oba światy',
 'vandroiy używa przenieść gabriela równoległego świata',
 'czasie biskup mainz johann adam von bicken brat jakob rządca falk von kronberg drodze rzymu zamierzają spotkać papieżem clemensem glory of rome',
 'nios

In [343]:
tfidf = Pipeline([
    ('count', CountVectorizer()),
    ('tfidf', TfidfTransformer(norm = None, sublinear_tf = False, smooth_idf = False))
])

In [344]:
centroid_vector_all = tfidf.fit_transform(sentences).toarray().sum(axis = 0)
centroid_vector_all = np.divide(centroid_vector_all, centroid_vector_all.max())


In [345]:
RELEVANT_VECTOR_CUTOFF_RATIO = 0.3

In [346]:
relevant_vector_indices = np.where(centroid_vector_all > RELEVANT_VECTOR_CUTOFF_RATIO)[0]

In [347]:
features = tfidf['count'].get_feature_names_out()
word_list = list(np.array(features)[relevant_vector_indices])

In [348]:
word_list

['anny',
 'annę',
 'avantasii',
 'brata',
 'czym',
 'druid',
 'elderane',
 'falk',
 'gabriel',
 'gabriela',
 'gabrielowi',
 'jakoba',
 'kielich',
 'klasztoru',
 'krasnoluda',
 'kronberg',
 'laymann',
 'mainz',
 'oba',
 'of',
 'opowiada',
 'pieczęci',
 'równoległego',
 'the',
 'uciec',
 'udaje',
 'uwolnić',
 'vandroiy',
 'von',
 'wieży',
 'zakrada',
 'zostaje',
 'świata',
 'światy']

In [349]:
all_words = [word_tokenize(sent) for sent in sentences]

all_words_flattened = reduce(operator.concat, all_words)

model = Word2Vec(all_words, window=2, size=100, sg = 1, min_count=1)

model_lookup = dict()
for word in all_words_flattened:
    model_lookup[word] = model.wv[word]

In [350]:
def make_vector_representation(words, model_lookup, model):
    representation = np.zeros(model.vector_size, dtype='float32')

    for word in words:
        if word in model_lookup.keys():
            representation += model_lookup[word]
    
    representation = np.divide(representation, len(words))

    return representation

In [351]:
centroid_vector = make_vector_representation(word_list, model_lookup, model)

In [352]:
representation = make_vector_representation(all_words_flattened, model_lookup, model)

In [353]:
representation

array([-1.49269559e-04,  4.59299154e-05, -1.55160509e-04,  1.61454009e-04,
       -1.17728159e-04,  1.20887657e-04, -2.01393865e-04,  1.17143943e-04,
        1.71534994e-04, -2.32780643e-04, -9.50396498e-05, -6.06605434e-04,
       -2.63876922e-04, -6.56531847e-05, -1.36236078e-04,  4.42116980e-05,
       -3.97881551e-04,  2.45767733e-04,  1.57000919e-04,  8.16823085e-05,
       -2.81049324e-05,  3.00926738e-04,  2.75014696e-04,  1.18664313e-04,
        2.20297719e-04, -1.65483914e-04,  8.89468211e-05,  2.98165869e-05,
       -1.23989856e-04,  2.15513410e-05, -2.16103275e-04,  1.77226961e-04,
        1.04506646e-04, -7.18528827e-05, -4.89952254e-05,  3.07196256e-04,
        5.92891010e-05,  2.26513963e-04,  1.91359446e-04, -1.18889133e-04,
        1.74621033e-04, -9.07594585e-05, -1.72945802e-04,  7.90878694e-05,
       -2.90512198e-05,  3.43407773e-05,  2.23394440e-04,  4.09848872e-05,
        5.49167489e-05, -1.28661908e-04, -1.43701021e-04,  1.91674946e-04,
       -5.19101159e-04,  

In [354]:
sent_scores = dict()
for n, sentence in enumerate(sentences):
    words = sentence.split()

    sentence_vector = make_vector_representation(words, model_lookup, model)

    score = cos_sim(sentence_vector, centroid_vector)
    sent_scores[n] = [score, sentences[n], sentence_vector, raw_sentences[n]]

sent_scores_sort = sorted(sent_scores.items(), key = lambda item: item[1][0], reverse=True)

In [355]:
sent_scores_sort

[(9,
  [0.3735139848735489,
   'vandroiy używa przenieść gabriela równoległego świata',
   array([ 4.8727225e-04, -5.3777552e-04, -6.3842809e-04, -1.2910570e-03,
           4.9432472e-04,  1.2408878e-03, -5.9562572e-04, -9.7090472e-04,
          -1.5653623e-03, -1.8471727e-03, -1.1421329e-04, -2.2149477e-03,
           5.8571686e-04, -1.0595856e-03,  2.2549019e-04,  1.4088178e-03,
          -1.1563725e-03,  1.3923375e-03,  1.7445091e-03,  1.6449094e-03,
           1.7185850e-03,  6.4844225e-04,  1.5144286e-03, -1.1582322e-03,
           1.7218845e-03,  4.3200221e-04,  2.9909940e-04,  8.3820213e-04,
           5.1371753e-04, -4.1554283e-04, -1.0894507e-03,  2.2189179e-03,
           1.0567376e-04, -4.9903319e-04, -2.6577476e-03, -5.1252282e-04,
           1.8196801e-03,  1.9553646e-03,  8.5204357e-04, -2.4464703e-04,
           1.0179736e-03, -1.2815302e-03, -8.2251307e-04, -1.4857032e-03,
          -3.7816141e-04,  1.1196822e-03,  5.5480905e-05, -1.2441521e-03,
          -3.8732821e-04

In [356]:
COS_SIM_CUT_OFF = 0.75

In [357]:
for s in sent_scores_sort:
    count = 0
    sentences_summary = []
    #Handle redundancy
    for s in sent_scores_sort:
        if count > 100:
            break
        include_flag = True
        for ps in sentences_summary:
            sim = cos_sim(s[1][2], ps[1][2])
            if sim > COS_SIM_CUT_OFF:
                include_flag = False
        if include_flag:
            sentences_summary.append(s)
            count += len(s[1][1].split())
    
    sentences_summary = sorted(sentences_summary, key=lambda el: el[0], reverse=False)

In [358]:
summarized_sents = []
for n, sent in enumerate(sentences_summary):
    summarized_sents.append(sentences_summary[n][1][3])

summarized_sents

['\nGłównym bohaterem jest Gabriel Laymann, nowicjusz klasztoru dominikanów w Mainz',
 'Zostaje przyłapany przez swojego nauczyciela brata Jakoba i uwięziony w lochu',
 'Główny bohater spotyka tam starszego człowieka Lugaida Vandroiy, który przedstawia mu się jako druid (Reach Out For The Light)',
 'Spotkany człowiek opowiada Gabrielowi o innym zagrożonym wymiarze – Avantasii',
 'Razem udaje im się uciec z lochów (Breaking Away) po czym Vandroiy zabiera Gabriela do kamieniołomu, gdzie ukryty jest portal łączący oba światy',
 'Vandroiy używa go by przenieść Gabriela do równoległego świata',
 'Gdy Gabriel dostaje się do równoległego świata jest powitany przez dwóch jego mieszkańców elfa Elderana oraz krasnoluda Regrina (Inside)',
 'Jednak Gabriel nie jest usatysfakcjonowany',
 'Elderane opowiada Gabrielowi o złotym kielichu ukrytym w rzymskich katakumbach',
 'Przebudzona bestia zabija jednak krasnoluda, Gabrielowi udaje się uciec',
 'Gabriel wraca do Vandroiya, który czekał na niego',
 '

In [359]:
def display_highlights(raw_sentences, summarized_sents):
    final = []
    for sent in raw_sentences:
        if sent not in summarized_sents:
            final += sent
        else:
            final += f'<span style="background-color:rgba(255,215,0,0.3);"> {sent} </span>'
    display(HTML(''.join([elem for elem in final])))

In [360]:
display_highlights(raw_sentences, summarized_sents)